In [1]:
from paddle import fluid
import numpy as np

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


In [2]:
train_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype='float32')
y_true = np.array([[1], [0], [0], [1]], dtype='float32')


In [3]:
import paddle
paddle.enable_static()

In [4]:
x = fluid.data(name='x', shape=[None, 2], dtype='float32')

## 隐藏层1
hidden = fluid.layers.fc(input=x, size=4, act='tanh')

## 隐藏层2
hidden2 = fluid.layers.fc(input=hidden, size=4, act='tanh')  # 填写缺失部分

## 输出层
y_predict = fluid.layers.fc(input=hidden2, size=1, act='tanh') 

In [5]:
y = fluid.data(name='y', shape=[None, 1], dtype='float32')
cost = fluid.layers.square_error_cost(input=y_predict, label=y)
avg_cost = fluid.layers.mean(cost)

In [6]:
# 优化方法(优化器)(要求：优化器采用SGD优化器)
sgd_optimizer = fluid.optimizer.SGD(learning_rate=0.1)  # 填写缺失部分
optimize_ops, params_grads = sgd_optimizer.minimize(avg_cost)


In [7]:
cpu = fluid.CPUPlace()
exe = fluid.Executor(cpu)
exe.run(fluid.default_startup_program())

[]

In [11]:
for i in range(1000):
    out_predict, out_cost = exe.run(
        program=fluid.default_main_program(),
        feed={'x': train_data, 'y': y_true},
        fetch_list=[y_predict, avg_cost])

    print("训练轮次:%d, Cost:%0.5f" % (i, out_cost[0]))
print("王磊202231060435")

In [12]:
test_data = np.array([[0, 0], [1, 1], [1, 0], [0, 1]], dtype='float32')

test_result = exe.run(
    program=fluid.default_main_program(),
    feed={'x': test_data, 'y': y_true},
    fetch_list=[y_predict])

In [13]:
print(np.round(test_result).astype(int))

[[[1]
  [1]
  [0]
  [0]]]
